In [10]:
from src.data import _load_readmission_dataset, _load_simData
from src.helper_functions import _prepare_rt_tensors, _prepare_dataloaders
from src.models import *
from src.losses import *
from src.training import *
from src.metrics import *

In [11]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
x, t, e, d = _load_readmission_dataset(sequential=True)

/home/ayush/DeepReEvent/src/data.py:47: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data['max_time'] = data.groupby('id')['t.stop'].transform(max)


In [13]:
readmission_tensor = _prepare_rt_tensors(x, t, e, d)
locals().update(readmission_tensor) # create variables from dictionary

In [14]:
# model and training parameters

input_size = len(x[0]) 
output_size = int(max(t))
hidden_size = 8   # Number of units in the RNN layer

model = SimpleGRU(input_size, hidden_size, output_size, 1, 0.1)

# Instantiate the model

num_epochs = 10000
patience = 3  # Number of epochs to wait for improvement before stopping
best_val_loss = float('inf')
wait = 0
loss_function = recurrent_loss

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),  lr=1e-4, weight_decay=1e-7)

In [15]:
#prepare data loaders

train_dataloader, val_dataloader, test_dataloader = _prepare_dataloaders(readmission_tensor, batch_size=32)

In [16]:
train_validate_recurrent_model(model, train_dataloader, val_dataloader, loss_function, optimizer, num_epochs, patience, print_every=10)

Epoch: 0   Training loss: 2.8611   Validation loss: 2.9589


Epoch: 10   Training loss: 2.8668   Validation loss: 2.9581
Epoch: 20   Training loss: 2.8403   Validation loss: 2.9565
Early stopping


In [17]:
test_predictions = model(x_test).squeeze(-1)

recurrent_predictions = test_predictions.squeeze(-1)

In [18]:
import torch
recurrent_cindex(recurrent_predictions, e_test, t_test, 12)

{'25th Quantile CI': 0.7151564404713531,
 '50th Quantile CI': 0.553623188405797,
 '75th Quantile CI': 0.553623188405797}

In [ ]:
nsamples = 10
survival_cis = []
recurrent_cis = []
for sample in range(0, nsamples): 
    print("sample: ",sample)
    readmission_tensor = _prepare_rt_tensors(x, t, e, d)
    locals().update(readmission_tensor) # create variables from dictionary

    train_dataloader, val_dataloader, test_dataloader = _prepare_dataloaders(readmission_tensor, batch_size=32)

    input_size = len(x[0]) 
    output_size = int(max(t))
    hidden_size = 8   # Number of units in the RNN layer

    model = SimpleLSTM(input_size, hidden_size, output_size, 2, 0.1)

    # Instantiate the model

    num_epochs = 10000
    patience = 3  # Number of epochs to wait for improvement before stopping
    best_val_loss = float('inf')
    wait = 0
    loss_function = recurrent_loss

    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(),  lr=1e-4, weight_decay=1e-7)
    train_validate_recurrent_model(model, train_dataloader, val_dataloader, loss_function, optimizer, num_epochs, patience, print_every=1000)
    test_predictions = model(x_test).squeeze(-1)

    survival_predictions = test_predictions[:, :, 0:1].squeeze(-1)
    recurrent_predictions = test_predictions[:, :, 1:2].squeeze(-1)
    recurrent_cis.append(recurrent_cindex(recurrent_predictions, e_test, t_test, 12))